In [1]:
import pandas as pd
import numpy as np
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_distances
from sklearn_extra.cluster import KMedoids
from sklearn.neighbors import LocalOutlierFactor
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import string
import plotly.express as px

# Descargar stopwords
nltk.download('stopwords')

# Inicializar el stemmer
stemmer = PorterStemmer()

# Función de preprocesamiento del texto
def preprocess_text(text):
    stop_words = set(stopwords.words('english'))  # Conjunto de stop words en inglés
    text = text.lower()  # Convertir el texto a minúsculas
    text = ''.join([char for char in text if char not in string.punctuation])  # Eliminar la puntuación
    words = text.split()
    words = [word for word in words if word not in stop_words]  # Eliminar stop words
    words = [stemmer.stem(word) for word in words]  # Aplicar stemming
    text = ' '.join(words)
    return text

# Leer el archivo CSV con codificación ISO-8859-1
def read_csv(file_path, num_rows=None):
    df = pd.read_csv(file_path, nrows=num_rows, encoding='ISO-8859-1')
    return df

# Convertir los textos a vectores TF-IDF y obtener la palabra más representativa
def get_top_word(text, vectorizer):
    tfidf_matrix = vectorizer.transform([text])  # Transformar el texto en una matriz TF-IDF
    feature_array = np.array(vectorizer.get_feature_names_out())  # Obtener los nombres de las características (palabras)
    tfidf_sorting = np.argsort(tfidf_matrix.toarray()).flatten()[::-1]  # Ordenar los puntajes TF-IDF en orden descendente
    top_word = feature_array[tfidf_sorting][0]  # Obtener la palabra con el puntaje más alto
    return top_word

# Función para eliminar outliers usando Local Outlier Factor (LOF)
def remove_outliers(X, n_neighbors=20):
    lof = LocalOutlierFactor(n_neighbors=n_neighbors)
    y_pred = lof.fit_predict(X)
    return y_pred == 1

# Función principal para realizar el análisis PCA
def pca_analysis(file_path, num_rows=None, remove_outliers_option=False):
    # Leer el archivo CSV
    df = read_csv(file_path, num_rows)
    
    # Preprocesar los textos
    df['cleaned_text'] = df['text'].apply(preprocess_text)
    
    # Convertir los textos a vectores TF-IDF
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(df['cleaned_text']).toarray()
    
    # Obtener la palabra más representativa
    df['top_word'] = df['cleaned_text'].apply(lambda x: get_top_word(x, vectorizer))
    
    # Estandarizar los datos
    scaler = StandardScaler()
    X_scaled = scaler.fit_transform(X)
    
    # Opcional: Eliminar outliers
    if remove_outliers_option:
        inliers = remove_outliers(X_scaled)
        X_scaled = X_scaled[inliers]
        df = df[inliers]
    
    # Aplicar PCA con tres componentes
    pca = PCA(n_components=3)
    principal_components = pca.fit_transform(X_scaled)
    
    df_pca = pd.DataFrame(data=principal_components, columns=['PC1', 'PC2', 'PC3'])
    df_pca['top_word'] = df['top_word'].values
    
    # Recalcular la matriz de distancias después de eliminar outliers
    distance_matrix = cosine_distances(X_scaled)
    
    # Clustering con K-Medoids y distancia de similitud coseno
    kmedoids = KMedoids(n_clusters=5, metric='precomputed', random_state=42)
    clusters = kmedoids.fit_predict(distance_matrix)
    df_pca['cluster'] = clusters
    
    # Visualización interactiva en 3D con plotly
    fig = px.scatter_3d(df_pca, x='PC1', y='PC2', z='PC3', color='cluster', text='top_word',
                        title='PCA of Tweet Data in 3D', labels={'PC1': 'Principal Component 1', 'PC2': 'Principal Component 2', 'PC3': 'Principal Component 3'})
    
    fig.update_traces(marker=dict(size=5), selector=dict(mode='markers'))
    fig.update_layout(scene=dict(
                        xaxis_title='Principal Component 1',
                        yaxis_title='Principal Component 2',
                        zaxis_title='Principal Component 3'),
                      margin=dict(l=0, r=0, b=0, t=40))
    
    # Guardar el gráfico interactivo en un archivo HTML
    fig.write_html("pca_tweet_data.html")
    
    fig.show()

# Ejemplo de uso
file_path = 'C:/Users/nicol/OneDrive/MIA/Cursos/Aprendizaje Sup/Proyecto de Aplicacion/REPO GITHUB/proyecto-final-machine-learning/dataset/train.csv'  # Reemplazar con la ruta a tu archivo CSV
num_rows = 100  # Número de filas a considerar para el análisis, None para usar todas las filas
remove_outliers_option = True  # Cambiar a False para no eliminar outliers

pca_analysis(file_path, num_rows, remove_outliers_option)


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicol\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
